GeoDatenErfassung_HS_Mainz
# Übung 2 – Graffiti Mapping in Mainz
Prof. Dr. Yu Feng

---

## Überblick

Diese Übung führt Sie durch einen kompletten Workflow zur Detektion und Kartierung von Graffiti anhand von Street‑View‑Bildern:

- Street‑View‑Bilder (Mapillary) für ein Untersuchungsgebiet abrufen
- Bilder lokal zwischenspeichern und per YOLOv5 (Kommandozeile) auf Graffiti detektieren
- Ergebnisse als thematisierte Geodaten auf einer interaktiven Karte visualisieren

---

## Lernziele

- Arbeiten mit Koordinatenreferenzsystemen (ETRS89 / UTM Zone 32N, EPSG:25832)
- Nutzung der Mapillary Graph API zur Datenakquise
- CLI‑basierte Inferenz mit YOLOv5 und Auswertung der Textausgaben (labels/*.txt)
- Kartographische Visualisierung (folium) mit inhaltlich codierten Markern und Popups

---

## Voraussetzungen

- Gültiges Mapillary API‑Token (als Umgebungsvariable `MAPILLARY_TOKEN`)
- Python‑Umgebung mit `requests`, `pyproj`, `folium`, `PIL` (Pillow); Internetzugang
- Für die Detektion: `git` und `pip` verfügbar (zum Laden von `yolov5` und Abhängigkeiten)
- YOLOv5‑Modell (Datei `model/best.pt` im Projektverzeichnis)

---

## Ablauf (Überblick)

1. **Untersuchungsgebiet festlegen**: Zentrum und BBox präzise in EPSG:25832 puffern und nach WGS84 zurückführen
2. **Bilder abrufen**: Mapillary Graph API nutzen, um alle Street‑View‑Bilder in der BBox zu laden
3. **Thumbnails speichern**: Bilder lokal im Ordner `images_dl/` ablegen
4. **Detektion ausführen**: YOLOv5 per Kommandozeile auf `images_dl/` laufen lassen; Ausgaben in `runs/graffiti_cli/exp/`
5. **Ergebnisse parsen**: `labels/*.txt` einlesen und in `has_graffiti` bzw. `results_by_id` strukturieren
6. **Assets vorbereiten**: Bilder komprimieren (geringere Dateigröße für HTML‑Export)
7. **Karte erstellen**: Interaktive Folium‑Karte mit farbcodierten Markern (Rot = Graffiti, Grün = sauber) und Popups mit annotierten Bildern

---

## Aufgaben (bearbeitbar/prüfbar)

- **A1.** Variieren Sie den Suchradius (z. B. 100 m, 300 m, 500 m) und vergleichen Sie die Bildanzahl.
- **A2.** Führen Sie die CLI‑Detektion aus und protokollieren Sie die Anzahl gefundener Graffiti‑Bilder.
- **A3.** Passen Sie die Konfidenzschwelle (`--conf`) an (z. B. 0.15/0.25/0.35) und diskutieren Sie Präzision/Recall.

---

## Hinweise

- **Ausgabeverzeichnisse**: Annotierte Bilder liegen nach der Detektion in `runs/graffiti_cli/exp/`; die Karte nutzt diese automatisch in den Popups.
- **Leere Ergebnisse**: Falls keine Bilder gefunden werden, vergrößern Sie den Radius oder wechseln Sie an einen anderen Ort (z. B. Innenstadt, Bahnhofsbereich).

## Schritt 0: Google Drive einbinden (nur für Google Colab)

Wenn Sie dieses Notebook in Google Colab ausführen, wird hier Ihr Google Drive eingebunden. Dies ermöglicht den Zugriff auf das YOLOv5-Modell und weitere Projektdateien.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

project_base_path = '/content/drive/MyDrive/Teaching_HS/P0002:1_Datenerfassung/github/GeoDatenErfassung25_HS_Mainz/Uebung2'
print(f"Project base path set to: {project_base_path}")

Mounted at /content/drive
Project base path set to: /content/drive/MyDrive/Teaching_HS/P0002:1_Datenerfassung/github/GeoDatenErfassung25_HS_Mainz/Uebung2


## Schritt 1: API-Token konfigurieren

Mapillary benötigt ein gültiges Access Token. Sie können sich kostenlos bei [Mapillary](https://www.mapillary.com/developer) registrieren und ein Token generieren.

In [ ]:
# -------------------------
# 0) Konfiguration: Mapillary Token aus Umgebungsvariable
# -------------------------
import os
import json
import math
import requests
from pyproj import Transformer

# Empfohlen: Umgebungsvariable setzen: export MAPILLARY_TOKEN="..."
# oder in Google Colab: userdata.get('MAPILLARY_TOKEN')

# TODO: Temporär im Code eintragen 
TOKEN = "PASTE_YOUR_MAPILLARY_TOKEN_HERE"  

if not TOKEN:
    raise RuntimeError("MAPILLARY_TOKEN fehlt. Bitte als Umgebungsvariable setzen oder temporär im Code eintragen.")

## Schritt 2: Untersuchungsgebiet festlegen und BBox berechnen

Das Untersuchungsgebiet wird durch einen Mittelpunkt (Lat/Lon) und einen Radius definiert. Die BoundingBox (BBox) wird präzise in EPSG:25832 (UTM Zone 32N) gepuffert, um gleichmäßige Abstände in Metern zu garantieren.

In [ ]:
# -------------------------
# 1) Untersuchungsgebiet (Mainz Rheingauwall) + BBox in EPSG:25832
# -------------------------
# Zentrum des Untersuchungsgebiets (anpassbar)
center_lat = 50.012635951893
center_lon = 8.244556440625274

# Radius in Metern (z.B. 300/500/1000)
# Hinweis: Kleiner Radius = weniger Bilder, aber schneller; größerer Radius = mehr Bilder
radius_m = 10

transformer_to_utm = Transformer.from_crs("EPSG:4326", "EPSG:25832", always_xy=True)
transformer_to_wgs = Transformer.from_crs("EPSG:25832", "EPSG:4326", always_xy=True)

def bbox_from_center(lat, lon, radius_m):
    """
    TODO: Erzeugt eine BoundingBox (BBox) um einen Mittelpunkt.
    """

    return [west, south, east, north]

bbox = bbox_from_center(center_lat, center_lon, radius_m)
print("BBox [west, south, east, north]:", bbox)
print(f"Untersuchungsgebiet: Zentrum ({center_lat}, {center_lon}), Radius: {radius_m}m")

BBox [west, south, east, north]: [8.241736967458799, 50.01081909766579, 8.247375705467897, 50.01445274072047]
Untersuchungsgebiet: Zentrum (50.012635951893, 8.244556440625274), Radius: 200m


## Schritt 3: Street-View-Bilder von Mapillary abrufen

Die Mapillary Graph API wird verwendet, um alle verfügbaren Street-View-Bilder innerhalb der BBox abzurufen. Die ersten 24 Bilder werden als Thumbnail-Galerie angezeigt.

In [4]:
# -------------------------
# 3) Bilder innerhalb der BBox abrufen und kurz anzeigen
# -------------------------
import requests
import time
from IPython.display import display, HTML

BASE = "https://graph.mapillary.com"
IMG_FIELDS = "id,thumb_1024_url,thumb_640_url,captured_at,geometry,compass_angle,computed_compass_angle"

def fetch_images_in_bbox(bbox, limit=None, per_page=1000, debug=False, sleep=0.05):
    """
    Ruft alle Mapillary-Bilder innerhalb der BBox ab.
    """
    west, south, east, north = bbox

    params = {
        "access_token": TOKEN,
        "bbox": f"{west},{south},{east},{north}",
        "fields": IMG_FIELDS,
        "limit": per_page,
        "image_type": "photo",
    }

    url = f"{BASE}/images"
    out = []
    page = 0

    while True:
        page += 1
        r = requests.get(url, params=params, timeout=60)
        if r.status_code != 200:
            raise RuntimeError(f"Mapillary API fehlgeschlagen: {r.status_code}\n{r.text}")

        data = r.json()
        items = data.get("data", []) or []
        out.extend(items)

        next_url = (data.get("paging") or {}).get("next")

        if debug:
            print(f"Seite {page}: {len(items)} Elemente | Gesamt {len(out)} | next={bool(next_url)}")

        # Begrenzen, falls limit gesetzt
        if limit is not None and len(out) >= limit:
            out = out[:limit]
            break

        # Ende, falls keine nächste Seite
        if not next_url:
            break

        # Nächste Seite: Graph API liefert vollständige URL; params leeren
        url = next_url
        params = None

        # Kurze Pause gegen Rate-Limits
        if sleep:
            time.sleep(sleep)

    return out

# Bilder abrufen
images = fetch_images_in_bbox(bbox, limit=None, debug=True)
print(f"\nBilder geladen: {len(images)}")

# Thumbnail-Galerie anzeigen (erste 24 Bilder)
thumb_blocks = []
for img in images[:24]:
    thumb = img.get("thumb_1024_url") or img.get("thumb_640_url")
    ts = img.get("captured_at") or ""
    if thumb:
        thumb_blocks.append(
            f'<div style="margin:6px;text-align:center;">'
            f'<img src="{thumb}" style="width:200px;border-radius:6px;">'
            f'<div style="font-size:12px;color:#555;">{ts}</div>'
            f"</div>"
        )

if thumb_blocks:
    gallery = "".join(thumb_blocks)
    display(HTML(f'<div style="display:flex;flex-wrap:wrap;">{gallery}</div>'))
else:
    print("Keine Thumbnails verfügbar.")

Seite 1: 729 Elemente | Gesamt 729 | next=False

Bilder geladen: 729


## Schritt 4: Bilder lokal speichern

Alle gefundenen Bilder werden als Thumbnails (1024px oder 640px) lokal im Ordner `images_dl/` gespeichert. Dies ist notwendig für die nachfolgende Detektion mit YOLOv5.

In [5]:
# -------------------------
# 4) Bilder lokal herunterladen
# -------------------------
import os
from urllib.parse import urlparse
from tqdm.notebook import tqdm

def download_images_locally(images, out_dir="images_dl"):
    """
    Lädt Thumbnails lokal herunter und speichert Metadaten.
    """
    os.makedirs(out_dir, exist_ok=True)
    items = []

    for img in tqdm(images, desc="Downloading images"):
        g = img.get("geometry") or {}
        if g.get("type") != "Point":
            continue
        lon, lat = g.get("coordinates", [None, None])
        if lon is None or lat is None:
            continue
        url = img.get("thumb_1024_url") or img.get("thumb_640_url")
        if not url:
            continue
        img_id = img.get("id")
        ext = os.path.splitext(urlparse(url).path)[1] or ".jpg"
        out_path = os.path.join(out_dir, f"{img_id}{ext}")

        try:
            r = requests.get(url, timeout=60)
            if r.status_code == 200:
                with open(out_path, "wb") as f:
                    f.write(r.content)
            else:
                continue
        except Exception:
            continue

        items.append({
            "id": img_id,
            "path": out_path,
            "lat": lat,
            "lon": lon,
            "heading": img.get("computed_compass_angle") if img.get("computed_compass_angle") is not None else img.get("compass_angle"),
            "remote_url": url,
        })
    return items

dl_items = download_images_locally(images, out_dir="images_dl")
print(f"Downloaded locally: {len(dl_items)}")

Downloaded locally: 729


## Schritt 5: YOLOv5 vorbereiten

Das YOLOv5-Repository wird geklont (falls noch nicht vorhanden) und die Abhängigkeiten werden installiert. Stellen Sie sicher, dass die Modell-Datei `model/best.pt` verfügbar ist.

Quelle: https://huggingface.co/BinKhoaLe1812/Graffiti-Dectection-YOLOv6m

In [6]:
# -------------------------
# 5) YOLOv5 per Kommandozeile (detect.py) vorbereiten
# -------------------------
import os
import sys
import subprocess
import shlex

# yolov5 klonen (falls nicht vorhanden)
repo_dir = os.path.join(os.getcwd(), "yolov5")
if not os.path.exists(repo_dir):
    cmd = "git clone https://github.com/ultralytics/yolov5"
    print(">", cmd)
    subprocess.run(shlex.split(cmd), check=True)
else:
    print("yolov5 bereits vorhanden; überspringe Klonen.")

# Abhängigkeiten installieren (idempotent)
req_path = os.path.join(repo_dir, "requirements.txt")
if os.path.exists(req_path):
    cmd = f"{sys.executable} -m pip install -r {req_path}"
    print(">", cmd)
    subprocess.run(shlex.split(cmd), check=False)

# Gewichte-Pfad wählen (lokal oder project_base_path, falls vorhanden)
weights_candidates = [os.path.join("model", "best.pt")]
if "project_base_path" in globals():
    cand = os.path.join(project_base_path, "model", "best.pt")
    if os.path.exists(cand):
        weights_candidates.insert(0, cand)

weights = None
for w in weights_candidates:
    if os.path.exists(w):
        weights = w
        break

if not weights:
    raise RuntimeError("Kein Modell gefunden (model/best.pt). Bitte bereitstellen.")

print(f"Verwende Modell: {weights}")

source = os.path.join("images_dl")
project = os.path.join("runs", "graffiti_cli")
name = "exp"

if not os.path.exists(source):
    raise RuntimeError("Verzeichnis images_dl nicht gefunden. Bitte zuerst Thumbnails herunterladen.")

print(f"Quellverzeichnis: {source}")
print(f"Ausgabeverzeichnis: {project}/{name}")

> git clone https://github.com/ultralytics/yolov5
> /usr/bin/python3 -m pip install -r /content/yolov5/requirements.txt
Verwende Modell: /content/drive/MyDrive/Teaching_HS/P0002:1_Datenerfassung/github/GeoDatenErfassung25_HS_Mainz/Uebung2/model/best.pt
Quellverzeichnis: images_dl
Ausgabeverzeichnis: runs/graffiti_cli/exp


## Schritt 6: Graffiti-Detektion ausführen

YOLOv5 wird per Kommandozeile (`detect.py`) auf alle heruntergeladenen Bilder angewendet. Die Ergebnisse (annotierte Bilder und Label-Dateien) werden in `runs/graffiti_cli/exp/` gespeichert.

In [ ]:
# -------------------------
# 6) YOLOv5 Detektion ausführen (mit Live-Output)
# -------------------------
detect_py = os.path.join(repo_dir, "detect.py")

# TODO: Anpassbare Parameter conf und analysieren Sie die Ergebnisse

cmd = shlex.split(
    f"{sys.executable} {detect_py} "
    f"--weights {weights} --source {source} "
    f"--img 1280 --conf 0.25 "
    f"--save-txt --save-conf "
    f"--project {project} --name {name} --exist-ok"
)

process = subprocess.Popen(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

print("Starting YOLOv5 detection (live output):\n")
print("=" * 60)

for line in process.stdout:
    print(line, end="")

process.wait()

print("=" * 60)

if process.returncode != 0:
    raise RuntimeError(f"YOLOv5 detect.py failed with code {process.returncode}")

print("\n✓ Detection finished successfully.")

Starting YOLOv5 detection (live output):

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
detect: weights=['/content/drive/MyDrive/Teaching_HS/P0002:1_Datenerfassung/github/GeoDatenErfassung25_HS_Mainz/Uebung2/model/best.pt'], source=images_dl, data=yolov5/data/coco128.yaml, imgsz=[1280, 1280], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/graffiti_cli, name=exp, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-453-geed9bc19 Python-3.12.12 torch-2.9.0

## Schritt 7: Detektionsergebnisse parsen

Die Label-Dateien (`labels/*.txt`) im YOLO-Format werden eingelesen und ausgewertet. Für jedes Bild wird gespeichert, ob Graffiti gefunden wurde (`has_graffiti`) und wo sich die Detektionen befinden (`results_by_id`).

In [8]:
# -------------------------
# 7) Ausgaben von detect.py parsen (inkl. BBox-Koordinaten)
# -------------------------
import os
import glob

labels_dir = os.path.join("runs", "graffiti_cli", "exp", "labels")
results_by_id = {}
has_graffiti = {}

# Optional: Nur bestimmte Klassen-IDs als Graffiti werten (z.B. {0})
# None = alle Klassen zählen als Graffiti
GRAFFITI_CLASS_IDS = None

# Alle heruntergeladenen Bilder initialisieren
for item in dl_items:
    img_id = item["id"]
    results_by_id[img_id] = []
    has_graffiti[img_id] = False

# Label-Dateien parsen
if os.path.isdir(labels_dir):
    for txt in glob.glob(os.path.join(labels_dir, "*.txt")):
        base = os.path.basename(txt)
        img_stem = os.path.splitext(base)[0]  # Dateiname ohne Endung = Image-ID
        img_id = img_stem
        dets = []

        with open(txt, "r") as f:
            for line in f:
                parts = line.strip().split()
                # YOLO-Format: cls cx cy w h conf
                if len(parts) < 6:
                    continue

                cls_id = int(float(parts[0]))
                cx = float(parts[1])
                cy = float(parts[2])
                bw = float(parts[3])
                bh = float(parts[4])
                conf = float(parts[5])

                dets.append({
                    "cls": cls_id,
                    "label": str(cls_id),
                    "conf": conf,
                    "cx": cx,
                    "cy": cy,
                    "w": bw,
                    "h": bh,
                })

                # Prüfen, ob diese Klasse als Graffiti zählt
                if GRAFFITI_CLASS_IDS is None or cls_id in GRAFFITI_CLASS_IDS:
                    has_graffiti[img_id] = True

        results_by_id[img_id] = dets

    graffiti_count = sum(1 for v in has_graffiti.values() if v)
    print(f"✓ Parsing fertig: {graffiti_count} Bilder mit Graffiti gefunden")
    print(f"  Gesamt: {len(dl_items)} Bilder analysiert")
else:
    print(f"⚠ Labels-Verzeichnis fehlt: {labels_dir}")
    print("  Bitte zuerst die CLI-Detektion ausführen.")

✓ Parsing fertig: 310 Bilder mit Graffiti gefunden
  Gesamt: 729 Bilder analysiert


## Schritt 8: Karten-Assets vorbereiten (Kompression)

Um die HTML-Dateigröße zu reduzieren, werden alle Bilder komprimiert (auf max. 400px Breite, JPEG-Qualität 70%). Dies beschleunigt das Laden der interaktiven Karte erheblich.

In [9]:
# -------------------------
# 8) Karten-Assets lokal vorbereiten mit Kompression
# -------------------------
import os
import requests
from PIL import Image

def prepare_map_assets(items, out_dir="map_assets", max_width=400, quality=70):
    """
    Bereitet komprimierte Bilder für die Karte vor.
    """
    os.makedirs(out_dir, exist_ok=True)

    for item in items:
        img_id = item.get("id")
        if not img_id:
            continue

        basename = os.path.basename(item.get("path", f"{img_id}.jpg"))
        ext = os.path.splitext(basename)[1] or ".jpg"
        annot_rel = os.path.join("runs", "graffiti_cli", "exp", basename)
        target_path = os.path.join(out_dir, f"{img_id}{ext}")

        try:
            # Priorisiere annotiertes Bild, falls vorhanden
            if os.path.exists(annot_rel):
                src_img = annot_rel
            else:
                url = item.get("remote_url")
                if not url:
                    continue
                r = requests.get(url, timeout=60)
                if r.status_code != 200:
                    continue
                tmp = os.path.join(out_dir, f"_tmp_{img_id}{ext}")
                with open(tmp, "wb") as f:
                    f.write(r.content)
                src_img = tmp

            # Bild komprimieren
            with Image.open(src_img) as im:
                w, h = im.size
                if w > max_width:
                    new_h = int(h * max_width / w)
                    im = im.resize((max_width, new_h), Image.LANCZOS)
                im = im.convert("RGB")
                im.save(target_path, "JPEG", quality=quality, optimize=True)

            item["asset_path"] = target_path
        except Exception:
            continue

    return out_dir

assets_dir = prepare_map_assets(dl_items, max_width=400, quality=70)
print(f"✓ Lokale Karten-Assets vorbereitet (komprimiert): {assets_dir}")

✓ Lokale Karten-Assets vorbereitet (komprimiert): map_assets


## Schritt 9: Interaktive Karte erstellen

Die finale Karte wird mit Folium erstellt:
- **Rote Marker**: Bilder mit Graffiti-Detektionen
- **Grüne Marker**: Bilder ohne Graffiti
- **Popups**: Zeigen das annotierte Bild mit eingezeichneten Bounding Boxes

Die Karte wird als `mainz_graffiti_karte.html` gespeichert und kann im Browser geöffnet werden.

In [10]:
# -------------------------
# Interaktive Karte (ein Symbol: Punkt + Keil, klickbar, Hover-Hervorhebung)
# Verwendet ausschließlich komprimierte Assets aus assets_dir
# -------------------------
import os
import math
import base64
import folium
from folium import Element
from IPython.display import display, HTML


# ---------- Popup: nur Bild + Text (keine Boxen) ----------
def build_overlay_html(img_path, img_id, dets, use_base64=True):
    """
    Popup nutzt ausschließlich lokale, komprimierte Bilder aus assets_dir.
    """
    if not img_path or not os.path.exists(img_path):
        return (
            "<div style='width:220px;'>"
            f"<b>ID:</b> {img_id}<br>"
            "<span style='color:#b00;'>Kein Asset in assets_dir gefunden.</span>"
            "</div>"
        )

    img_display = img_path
    if use_base64:
        try:
            with open(img_path, "rb") as f:
                img_b64 = base64.b64encode(f.read()).decode("utf-8")
            img_display = f"data:image/jpeg;base64,{img_b64}"
        except Exception:
            img_display = img_path

    det_lines = []
    for d in (dets or [])[:10]:
        if d.get("label") and d.get("conf") is not None:
            try:
                det_lines.append(f"{d['label']} ({float(d['conf']):.2f})")
            except Exception:
                det_lines.append(f"{d['label']}")

    html = "<div style='width:260px;'>"
    html += (
        "<div style='width:240px;height:240px;'>"
        f"<img src='{img_display}' "
        "style='width:240px;height:240px;object-fit:contain;"
        "border-radius:6px;display:block;'>"
        "</div>"
    )
    html += f"<div style='font-size:12px;margin-top:6px;'><b>ID:</b> {img_id}</div>"

    if det_lines:
        html += (
            "<div style='font-size:12px;margin-top:6px;'>"
            "<b>Detektionen:</b><br>"
            + "<br>".join(det_lines)
            + "</div>"
        )

    html += "</div>"
    return html


# ---------- Kombiniertes Symbol: Punkt + Keil ----------
def make_point_wedge_icon(heading_deg, color="#43A047",
                          dot_r=5, wedge_len=22, half_angle_deg=18):
    if heading_deg is None:
        heading_deg = 0.0

    dot_r = float(dot_r)
    L = float(wedge_len)
    a = math.radians(float(half_angle_deg))

    x1 = math.sin(a) * L
    y1 = -math.cos(a) * L
    x2 = -math.sin(a) * L
    y2 = -math.cos(a) * L

    pad = 6.0
    min_x = min(x1, x2, 0.0, -dot_r) - pad
    max_x = max(x1, x2, 0.0,  dot_r) + pad
    min_y = min(y1, y2, 0.0, -dot_r) - pad
    max_y = max(y1, y2, 0.0,  dot_r) + pad

    width = max_x - min_x
    height = max_y - min_y

    cx = -min_x
    cy = -min_y

    html = f"""
    <div class="pw-wrap" style="
        width:{width:.1f}px;
        height:{height:.1f}px;
        transform: rotate({float(heading_deg):.1f}deg);
        transform-origin: {cx:.1f}px {cy:.1f}px;
        pointer-events:auto;
    ">
      <svg width="{width:.1f}" height="{height:.1f}"
           style="display:block;pointer-events:none;">
        <polygon points="{cx:.1f},{cy:.1f}
                         {cx + x1:.1f},{cy + y1:.1f}
                         {cx + x2:.1f},{cy + y2:.1f}"
                 fill="{color}" fill-opacity="0.35"
                 stroke="{color}" stroke-width="2"/>
        <circle cx="{cx:.1f}" cy="{cy:.1f}" r="{dot_r:.1f}"
                fill="{color}" fill-opacity="0.95"
                stroke="{color}" stroke-width="2"/>
      </svg>
    </div>
    """

    return folium.DivIcon(
        html=html,
        icon_size=(width, height),
        icon_anchor=(cx, cy)
    )


# ---------- Karte ----------
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=18,
    max_zoom=20
)

layer = folium.FeatureGroup(name="Aufnahmen").add_to(m)
marker_names = []
HOVER_SCALE = 1.45

skipped = 0

for item in dl_items:
    # --- Koordinaten ---
    if item.get("lat") is None or item.get("lon") is None:
        continue
    try:
        lat = float(item["lat"])
        lon = float(item["lon"])
    except Exception:
        continue

    img_id = item.get("id", "")
    heading = item.get("heading")

    # --- NUR assets_dir ---
    asset_path = item.get("asset_path")
    if not asset_path or not asset_path.startswith(assets_dir) or not os.path.exists(asset_path):
        skipped += 1
        continue

    has_det = bool(has_graffiti.get(img_id))
    color = "#E53935" if has_det else "#43A047"

    dets = results_by_id.get(img_id) or []
    popup_html = build_overlay_html(asset_path, img_id, dets)

    icon = make_point_wedge_icon(
        heading_deg=float(heading) if heading is not None else 0.0,
        color=color
    )

    mk = folium.Marker(
        location=[lat, lon],
        icon=icon,
        tooltip=f"ID: {img_id}",
        popup=folium.Popup(popup_html, max_width=320)
    ).add_to(layer)

    marker_names.append(mk.get_name())

folium.LayerControl().add_to(m)

# ---------- Hover-Vergrößerung ----------
js = ["<script>"]
js.append("""
function _pw_set_scale(el, s){
  if(!el) return;
  el.style.transform = el.style.transform.replace(/ scale\\([^\\)]*\\)/,'');
  el.style.transform += ' scale(' + s + ')';
}
""")

for name in marker_names:
    js.append(f"""
    (function(){{
      var m = {name};
      m.on('add', function(){{
        var el = m.getElement();
        if(!el) return;
        var w = el.querySelector('.pw-wrap');
        if(!w) return;
        el.addEventListener('mouseenter', function(){{ _pw_set_scale(w, {HOVER_SCALE}); }});
        el.addEventListener('mouseleave', function(){{ _pw_set_scale(w, 1.0); }});
      }});
    }})();
    """)

js.append("</script>")
m.get_root().html.add_child(Element("\\n".join(js)))

out_html = project_base_path + "/mainz_graffiti_karte.html"
m.save(out_html)

print(f"Karte gespeichert: {out_html}")
print(f"Übersprungen (kein Asset in {assets_dir}): {skipped}")

display(HTML(m._repr_html_()))


Output hidden; open in https://colab.research.google.com to view.

## Zusatzaufgabe (Optional): Graffiti auf Gebäudewänden lokalisieren

**Ziel:** Graffiti-Objekte mithilfe von OSM-Gebäudedaten und Triangulation der nächstgelegenen Gebäudewand zuordnen.

**Schritte:**
1. OSM-Gebäude in der BBox abrufen (Overpass API)
2. Für jeden Graffiti-Punkt die nächstgelegene Gebäudewand berechnen
3. Vergleich: Kamerarichtung (Heading) ↔ Wandorientierung
4. Statistik: Auf welchen Häusern/Wänden am meisten Graffiti?

**Bonus:** Visualisierung auf der Karte (z. B. Gebäudepolygone mit Farbcodierung nach Graffiti-Dichte).
